In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
import astropy.units as u
# from icecream import ic, install

from astropaul.database import database_connection
import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [3]:
html_dir = "../../../../Observing Files/Fan Mountain 2025"
html.clear_directory(html_dir)
session = tlc.ObservingSession(
    ap.Observer(EarthLocation(lat="37:52:41.35", lon="-78:41:34.92", height=566 * u.m), timezone="utc", name="Fan Mountain")
)
session.add_day_range("2025-08-29", "2025-12-31")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

fan_targets = [
    # target of interest
    "TIC 344541836",
    # wide, bright QEBs
    "TIC 278465736",
    "TIC 470710327",
    "TIC 367448265",
    "TIC 317863971",
    "TIC 252980672",
    "TIC 159089190",
    "TIC 70664819",
    "TIC 286470992",
    "TIC 307119043",
    "TIC 200094011",
]

# fan_targets = [
#     # wide, bright QEBs
#     "TIC 278465736",
#     "TIC 470710327",
#     "TIC 367448265",
#     "TIC 376606423",
#     "TIC 317863971",
#     "TIC 159089190",
#     "TIC 70664819",
#     "TIC 286470992",
#     "TIC 78568780",
#     "TIC 200094011",
# ]

min_altitude = 35 * u.deg

with database_connection() as conn:
    name = "Fan Mountain SIDE Summer 2025"
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(fan_targets))),
        tlc.ancillary_data_from_tess,
        partial(
            tlc.add_observability,
            observing_session=session,
            calc_moon_distance=True,
            observability_threshold=(min_altitude, 80 * u.deg),
            time_resolution=1 * u.hour,
        ),
        partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
        tlc.add_tess_catalog_associations,
        tlc.add_lists,
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(
            tlc.add_system_configuration,
            table_name="DSSI Observations",
            time_column="Mid JD",
            eclipse_table="SIDE Observations",
        ),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        tlc.add_pepsi_evaluations,
        partial(tlc.add_system_configuration, table_name="PEPSI Observations", time_column="Mid JD"),
        partial(tlc.add_phase_events, observing_session=session, event_types=["Eclipse", "Not in Eclipse"]),
    ]
    tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, html_dir)
print(tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list = answer.target_list.assign(**new_cols)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list = answer.target_list.assign(**new_cols)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

Name: Fan Mountain SIDE Summer 2025
Criteria:
  lambda df: (df["Target Name"].isin(['TIC 344541836', 'TIC 278465736', 'TIC 470710327', 'TIC 367448265', 'TIC 317863971', 'TIC 252980672', 'TIC 159089190', 'TIC 70664819', 'TIC 286470992', 'TIC 307119043', 'TIC 200094011']))
  Observability calculated at Fan Mountain in 1.0 h intervals
  AltitudeConstraint: {'min': 35.0, 'max': 80.0, 'boolean_constraint': True}
  lambda df: (df["Observable Any Night"])
  
11 targets:
    11 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    Observable: (5, 500)
    List: (0, 16)
    Count: (5, 0)
Associated tables:
     125 rows,  2 columns: Lunar Phases
      54 rows,  3 columns: Catalog Membership
      66 rows,  2 columns: List Memberships
      34 rows,  7 columns: Ephemerides
      49 rows, 12 columns: DSSI Observations
      82 rows, 11 columns: Speckle Detections
      23 rows,  6 columns: SIDE Observations
      46 rows, 18 columns: PEPSI Observations
    7249 

In [4]:
tl.target_list

,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Target Source,Tic,Vmag,Teff,...,List Speckle Resolved,List Table 1 Targets,List Zasche 2022,List HQND 2025-09-30,List Resolved 2025-09-30,Num Ephemerides,Num DSSI Observations,Num Speckle Detections,Num SIDE Observations,Num PEPSI Observations
0,TIC 200094011,86.782958,0.298942,05:47:07.91,+00:17:56.19,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,200094011,9.7000,NaN,...,True,True,False,False,True,4,2,2,0,6
1,TIC 286470992,45.330708,60.572294,03:01:19.37,+60:34:20.26,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,286470992,10.3250,8693.00,...,True,True,True,False,True,3,1,4,1,2
2,TIC 307119043,14.827542,51.221642,00:59:18.61,+51:13:17.91,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,307119043,9.9400,7815.00,...,True,True,False,False,True,4,1,2,0,10
3,TIC 317863971,110.567500,3.031925,07:22:16.2,+03:01:54.93,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,317863971,10.3060,8506.00,...,True,True,False,False,True,4,2,4,2,2
4,TIC 344541836,317.850708,57.620411,21:11:24.17,+57:37:13.48,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,344541836,7.9000,NaN,...,True,True,False,False,True,4,10,18,6,2
5,TIC 367448265,78.382458,35.653053,05:13:31.79,+35:39:10.99,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,367448265,7.9000,9212.00,...,True,True,True,False,True,3,13,18,1,12
6,TIC 470710327,357.329042,61.962786,23:49:18.97,+61:57:46.03,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,470710327,9.6500,8986.00,...,True,True,False,False,True,3,8,16,4,12
7,TIC 278465736,324.800500,51.346631,21:39:12.12,+51:20:47.87,QuadEB,Kostov 2023 doi.org/10.1093/mnras/stad2947,278465736,10.2720,6650.00,...,False,False,False,False,True,3,9,18,9,0
8,TIC 159089190,88.512458,1.672725,05:54:02.99,+01:40:21.81,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,159089190,10.1130,NaN,...,False,False,True,False,True,2,1,0,0,0
9,TIC 252980672,121.407542,52.353000,08:05:37.81,+52:21:10.8,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,252980672,10.5362,7044.15,...,False,False,True,False,True,2,1,0,0,0


In [5]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
# pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, html_dir)
# pl.categorical_priorities[0]